# Lab1.4: Google news as a source of text

Copyright: Vrije Universiteit Amsterdam, Faculty of Humanities, CLTL

In this notebook, we use the Google news API to search for news: 

https://news.google.com/?hl=en-US&gl=US&ceid=US:en

We follow a similar approach as for Techcrunch and use BeautifulSoup to get the content. However, now we have to deal with the structure of the Google news output. We do the usual imports for making a request and process the output through BeautifulSoup. Obviously, Google uses a very different API than Techcrunch to do a search and the results also look differently.

In [1]:
from bs4 import BeautifulSoup
import requests


## 1. Making a request to the Goggle News API

The next cell shows how to make a URL for the Google news search API. This URL is a bit more complex than the one for Techcrunch. You need to pass in some variables in the REST API such as the query *q* and the language *gl*.

In [3]:
import requests
query='vaccines'

query = query.lower()
language='en'
region='us'
base_url = "http://news.google.com"
query_url = "{0}/?q={1}&hl={1}-{2}&gl={2}".format(base_url, query, language, region)
print("The query URL looks like:", query_url)


The query URL looks like: http://news.google.com/?q=vaccines&hl=vaccines-en&gl=en


To see what it does, you can copy the URL in a browser and inspect the resul. In the next cell, we use the *requests* package to do the same and store the result.

In [4]:
google_news_content= requests.get(query_url).content

The result is an HTML string. We can inspect the start of this string. Let's look at the first 1000 characters:

In [5]:
print(google_news_content[:1000])

b'<!doctype html><html lang="en" dir="ltr"><head><base href="https://news.google.com/"><meta name="referrer" content="origin"><link rel="canonical" href="https://news.google.com/search"><meta name="viewport" content="width=device-width,initial-scale=1,minimal-ui"><meta name="apple-itunes-app" content="app-id=459182288"><meta name="google-site-verification" content="AcBy5YFny2HQgVUCR18tO5YUTf6MpVlcJqGTd-a9-SI"><meta name="mobile-web-app-capable" content="yes"><meta name="apple-mobile-web-app-capable" content="yes"><meta name="application-name" content="News"><meta name="apple-mobile-web-app-title" content="News"><meta name="apple-mobile-web-app-status-bar-style" content="black"><meta name="theme-color" content="white"><meta name="msapplication-tap-highlight" content="no"><link rel="shortcut icon" href="https://lh3.googleusercontent.com/-DR60l-K8vnyi99NZovm9HlXyZwQ85GMDxiwJWzoasZYCUrPuUM_P_4Rb7ei03j-0nRs0c4F=w16" sizes="16x16"><link rel="shortcut icon" href="https://lh3.googleusercontent

## 2. Using BeatifulSoup to process the result

We are going to use BeautifulSoup again to flesh out the results. So let's first turn the HTML string returned by Google into a *soup* object.

In [6]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(google_news_content, 'html.parser')

We are not going into details about the specific structure of the Google news result but we define two specific functions that get the *author* of the news article and the actual *content*.

Analyse the two functions and try to understand the code. Note that if the format of the Google news output changes, also the code may have to be adapted. These functions use regular expressions: https://docs.python.org/3/library/re.html 

Regular expressions allow you to 'parse' strings using simple patterns. We need to import the *re* package to use regular expressions.

In [7]:
import re

#This function requires the HML string an input parameter
#It looks for the tag 'author' in meta properties and content properties


def parse_author(html):
    result = ""
    htmlSoup = BeautifulSoup(html, 'html.parser')
    # Try Parsing Author from Meta Tags
    author = htmlSoup.find('meta', attrs={'name': re.compile('author')}) or \
             htmlSoup.find('meta', property=re.compile('author', re.IGNORECASE))

    if author: result = author['content']
    else:  # Otherwise, try parsing Author from Text
        author = htmlSoup.find(attrs={'itemprop': 'author'}) or \
                 htmlSoup.find(attrs={'class': 'byline'})
        if author: result = author.text

    return re.sub(r"\s+", " ", re.sub("by ", "", result, flags=re.IGNORECASE)).strip()


#This function requires the HTML content of the result as an input parameter
#It returns the actual text content

def parse_news_text(html):

    htmlSoup = BeautifulSoup(html, 'html.parser')

    # Try to find Article Body by Semantic Tag
    article = htmlSoup.find('article')

    # Otherwise, try to find Article Body by Class Name (with the largest number of paragraphs)
    if not article:
        articles = htmlSoup.find_all(class_=re.compile('(body|article|main)', re.IGNORECASE))
        if articles:
            article = sorted(articles, key=lambda x: len(x.find_all('p')), reverse=True)[0]

    # Parse text from all Paragraphs
    text = []
    if article:
        for paragraph in [tag.text for tag in article.find_all('p')]:
            if re.findall("[.,!?]", paragraph):
                text.append(paragraph)
    text = re.sub(r"\s+", " ", " ".join(text))

    return text

The Google news output consists of *articles*. We therefore need code to iterate over all article data elements and collect the data. The next code shows how we aggregate the content and meta information for each article, where we apply the above functions to obtain the author and the text from the content obtained from the URL. We break the loop after the first iteration.

In [14]:
# Iterate over all Articles in Google News
base_url = "http://news.google.com"
articles = soup.find_all('article')
for i, article in enumerate(articles, 1):
    ## new_entry is going to contain the data for each article returned
    new_entry = []
    
    div, title, publisher = article.find_all('a') ### returning tuples for the elements 'div', 'title', 'publisher'
    time = re.sub("[Z\-:]", "", article.find('time').get('datetime'))
    print('time:', time)
    print('div:', div)
    print('title:', title)
    print('publisher:', publisher)
    
    article_redirect = "{}{}".format(base_url, title.get('href')[1:])  ### creates a redirect from the google base and the returned result
    print('article_redirect:', article_redirect) ### creates the real URL from the redirect
    
    article_url = requests.get(article_redirect).url  ###
    print('article_url:', article_url)
    print()    
    news_content= requests.get(article_url).content  ### we get the content from the url
    author = parse_author(news_content)
    print('Author:', author)
    news_text = parse_news_text(news_content)
    
    #new entry append in the order of the data frame
    new_entry += [article_url, time, language, author, publisher.text, title.text, news_text]
    print('Entry:', new_entry)
    break ### we stop the for-loop after the first iteration. We do the complete loop later on when saving the results


time: 20191030T170047
div: <a aria-hidden="true" class="VDXfz" href="./articles/CBMiXmh0dHBzOi8vd3d3LmNvbnRhZ2lvbmxpdmUuY29tL25ld3MvZ2Fwcy1pbi1wZWRpYXRyaWMtdmFjY2luYXRpb24tbGlua2VkLXdpdGgtdW5pbnN1cmVkLXN0YXR1cy3SAQA?hl=en-US&amp;gl=US&amp;ceid=US%3Aen" jsname="hXwDdf" tabindex="-1" target="_blank"></a>
title: <a class="DY5T1d" href="./articles/CBMiXmh0dHBzOi8vd3d3LmNvbnRhZ2lvbmxpdmUuY29tL25ld3MvZ2Fwcy1pbi1wZWRpYXRyaWMtdmFjY2luYXRpb24tbGlua2VkLXdpdGgtdW5pbnN1cmVkLXN0YXR1cy3SAQA?hl=en-US&amp;gl=US&amp;ceid=US%3Aen">Gaps in Pediatric Vaccination Linked With Uninsured Status</a>
publisher: <a class="wEwyrc AVN2gc uQIVzc Sksgp">Contagionlive.com</a>
article_redirect: http://news.google.com/articles/CBMiXmh0dHBzOi8vd3d3LmNvbnRhZ2lvbmxpdmUuY29tL25ld3MvZ2Fwcy1pbi1wZWRpYXRyaWMtdmFjY2luYXRpb24tbGlua2VkLXdpdGgtdW5pbnN1cmVkLXN0YXR1cy3SAQA?hl=en-US&gl=US&ceid=US%3Aen
article_url: https://www.contagionlive.com/news/gaps-in-pediatric-vaccination-linked-with-uninsured-status-

Author: 
Entry: ['https:

The first result is printed. With the author (possibly empty) and the search result.

## 3 Save the search results in a CSV file with the meta data

With above function, we can iterate over all the search results, get the author, get the URL and possibly get some other meta data such as the date, the language, the owner and a title. We could save this as a text file, as we did for the Textcrunch search but since we may get some metadata that we want to preserve as well, we can also create a CSV file (comma or tab separated values). CSV represent tables as strings. We can easliy load the data back as a table for further processing. This is what we are going to do next.

Now we want to store the results for the complete loop in a CSV file so that we can load it later on. To create the output as CSV data, we are going to use the `Pandas` package: https://pandas.pydata.org.

Please follow the instructions to install pandas locally:

* `conda install pandas`
* `python -m pip install --upgrade pandas`

Consult the documentation to learn more about the functionalities. Here we are going to use it to convert our list of featurures for a tweet to a CSV format.

We need to import *os* for writing to a file and *pandas* (after the install) for dealing with the data structure. Take your time to study the next bit of code so that you understand the individual steps. 

We need to define the columns for the result table. The data needs to be stored in the order of the columns. The next cell shows the columns in the order we want to get the data. If you change this order, you also need to present the data accordingly and change the code below.

In [10]:
COLS = ['url', 'created_at', 'lang', 'author',  'publisher', 'title', 'news_text']

Now we use the same article processing for-loop to obtain the data for each article and store the result in a *pandas* data frame that we call `all_news_dataframe`. You can see this as a database in memory that has the structure of a table, with our predefined columns and on each row the data we get for each article in the Google result.

So the data that we obtain for an article can be seen as a list of values that we assign to a *new_entry* variable as follows:

```
    new_entry += [article_url, time, language, author, publisher.text, title.text, news_text]
```

Given the columns and a *new_entry* list, we can use *pandas* to create a single row data frame as follows:
```
    single_article_dataframe = pd.DataFrame([new_entry], columns=COLS)
```

Next, we need to add all these *single_article_dataframes* over all articles. 

The next code does all this and stores the aggregated result in the variable *all_news_dataframe*.

In [15]:
import os
import pandas as pd

# We first define a data frame that we name 'all_news_dataframe' 
# with pandas imported as 'pd' using the columns list that we defined before.
# Basically, we tell pandas what data will be stored.
base_url = "http://news.google.com"
all_news_dataframe = pd.DataFrame(columns=COLS)

# Iterate over all Articles in Google News
articles = soup.find_all('article')
for i, article in enumerate(articles, 1):
    ## new_entry is going to contain the data for each article returned
    new_entry = []
    
    div, title, publisher = article.find_all('a')
    time = re.sub("[Z\-:]", "", article.find('time').get('datetime'))

    article_redirect = "{}{}".format(base_url, title.get('href')[1:])
    ### Since the request call may generate an error from the website it tries to reach
    ### We have to catch the error message so that we can continue with the next URL to obtain a result
    ### To handle the errors, we create a try and except block. If there is an error, we print it, otherwise we carry out commands
    try:
        article_url = requests.get(article_redirect).url
    
        news_content= requests.get(article_url).content
        author = parse_author(news_content)
        news_text = parse_news_text(news_content)
    
        #new entry append in the order of the data frame
        new_entry += [article_url, time, language, author, publisher.text, title.text, news_text]

        # We now completed appending all the possible values for this tweet.
        # We use the pandas framework imported as 'pd' to create a dataframe from the aggregated data in new_entry
        # We need to provide the columns COLS to tell pandas what value belongs to what.
        # Note that the data need to be aggregated in the same order as the names in COLS, otherwise values will get mixed up
        single_article_dataframe = pd.DataFrame([new_entry], columns=COLS)
        
        # single_tweet_dataframe now contains the data for a single tweet
        # next we add it to the data frame for all tweets 'all_tweets_dataframe'
        # check the pandas documentation if you want to know what ignore_index=True does to the data aggregation
        all_news_dataframe = all_news_dataframe.append(single_article_dataframe, ignore_index=True)
    except requests.exceptions as e:
        print("Error:" + str(e))

TypeError: catching classes that do not inherit from BaseException is not allowed

When running the code, you may see a whole batch of error messages. Most likely, these are due to websites that cannot be reached. Basically, we fail to collect the data from the site. Still, some of the sites could be reached. To make sure that our loop over the articles does not stop after the first error, we are use the *try* and *except* blocks so that each error is catched and printed but the loop continues to the next.

Our data frame basically is a table with columns and rows. We use the `shape` function to ask for the number of rows and columns.

In [16]:
print(all_news_dataframe.shape)

(58, 7)


Through the *pandas* framework, we can now save it to a CSV file.

In [17]:
# We define a file path to store the results as CSV. Make sure the folder 'googlenews_search_results' exists 
# or that you specify another path to an existing location. The 'news_results_<query>.csv' file will be created in that location.
csvFilePath='googlenews_search_results/news_results_'+query+'.csv'

# we now open the csvFile for appending our result
csvFile = open(csvFilePath,"w+")       
all_news_dataframe.to_csv(csvFile, columns=COLS, index=False)

You can open the CSV file in Anaconda Jupyter notebooks or in a spreadsheet program such as Excel.

Unfortunately, the Google News API is no longer maintained. It is still running but it is not known for how long:

https://medium.com/rakuten-rapidapi/top-10-best-news-apis-google-news-bloomberg-bing-news-and-more-bbf3e6e46af6


In the next notebook Lab1.5, we show how you can access other news sources directly.

## End of this notebook